In [1]:
!git clone https://github.com/Israel-Anaba/Movie_Review_Analysis.git

Cloning into 'Movie_Review_Analysis'...
remote: Enumerating objects: 1627, done.
remote: Counting objects: 100% (1627/1627), done.
remote: Compressing objects: 100% (1482/1482), done.
remote: Total 1627 (delta 149), reused 1575 (delta 125), pack-reused 0
Receiving objects: 100% (1627/1627), 33.12 MiB | 23.86 MiB/s, done.
Resolving deltas: 100% (149/149), done.


In [2]:
%cd Movie_Review_Analysis

/content/Movie_Review_Analysis


### **Package Installation and Importing Libraries**

In [3]:
!pip install datasets
!pip install torch
!pip install transformers
!pip install huggingface_hub
!pip install accelerate>=0.20.1
!pip install transformers[torch]
!pip install wordcloud
!pip install collections-extended
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import string
import re
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import accelerate
import sys
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
import torch
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, BertConfig
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from datasets import load_metric
from sklearn.metrics import mean_squared_error
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from collections import Counter
from huggingface_hub import notebook_login
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
# Disabe W&B
os.environ["WANDB_DISABLED"] = "true"

###**Loading and Preprocessing Data**

In [7]:
# Load the dataset and display some values
df = pd.read_csv('/content/Movie_Review_Analysis/Assets/Train.csv')

test = pd.read_csv('/content/Movie_Review_Analysis/Assets/Test.csv')

In [8]:
df.head()

,review_file,content,sentiment
0,3471_8.txt,Recently shown on cable tv the movie opens wit...,positive
1,9693_8.txt,I was very surprised with this film. I was tou...,positive
2,10801_1.txt,"Now, I'm one to watch movies that got poor rev...",negative
3,9592_8.txt,"This film came out 12 years years ago, and was...",positive
4,8514_7.txt,"When an orphanage manager goes on vacation, hi...",positive


In [9]:
test.head()

,review_file,content
0,0_10.txt,I went and saw this movie last night after bei...
1,0_2.txt,Once again Mr. Costner has dragged out a movie...
2,10000_4.txt,This is an example of why the majority of acti...
3,10000_7.txt,Actor turned director Bill Paxton follows up h...
4,10001_1.txt,"First of all I hate those moronic rappers, who..."


In [10]:
df.rename(columns={'content': 'review'}, inplace=True)

In [11]:
df['sentiment'].nunique

<bound method IndexOpsMixin.nunique of 0        positive
1        positive
2        negative
3        positive
4        positive
           ...   
24995    negative
24996    positive
24997    positive
24998    negative
24999    positive
Name: sentiment, Length: 25000, dtype: object>

In [12]:
df.drop('review_file', axis=1, inplace=True)

In [13]:
df.head()

,review,sentiment
0,Recently shown on cable tv the movie opens wit...,positive
1,I was very surprised with this film. I was tou...,positive
2,"Now, I'm one to watch movies that got poor rev...",negative
3,"This film came out 12 years years ago, and was...",positive
4,"When an orphanage manager goes on vacation, hi...",positive


In [14]:
# convert the 'safe_text' column to lowercase
df['review'] = df['review'].str.lower()

In [15]:
df

,review,sentiment
0,recently shown on cable tv the movie opens wit...,positive
1,i was very surprised with this film. i was tou...,positive
2,"now, i'm one to watch movies that got poor rev...",negative
3,"this film came out 12 years years ago, and was...",positive
4,"when an orphanage manager goes on vacation, hi...",positive
...,...,...
24995,"as with most of the reviewers, i saw this on s...",negative
24996,a have a female friend who is currently being ...,positive
24997,like a streetcar named desire (also directed b...,positive
24998,"as a native film professor, i can honestly say...",negative


In [16]:
def remove_user_url(text):
    cleaned_text = re.sub(r'\b\w*user\w*\b|\b\w*url\w*\b', '', text, flags=re.IGNORECASE)
    return cleaned_text

# Use .loc to explicitly modify the 'safe_text' column in the original DataFrame
df['review'] = df['review'].apply(lambda x: remove_user_url(x))

In [17]:
def remove_special_characters(text):
    # Remove special characters, punctuation, and digits
    cleaned_text = re.sub(r'[^\w\s]|[\d]+', '', text)
    return cleaned_text

# Apply the function to the 'safe_text' column
df['review'] = df['review'].apply(remove_special_characters)

# # convert the 'safe_text' column to lowercase
# df['review'] = df['review'].str.lower()

In [18]:
# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization on a text
def lemmatize_text(text):
    words = word_tokenize(text)  # Tokenize the text into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Apply lemmatization to the 'safe_text' column
df['review'] = df['review'].apply(lemmatize_text)

In [ ]:
#  # Initialize the Porter Stemmer
# stemmer = PorterStemmer()

# # Function to perform stemming on a text
# def stem_text(text):
#     words = word_tokenize(text)
#     stemmed_words = [stemmer.stem(word) for word in words]  # Stem each word
#     return ' '.join(stemmed_words)  # Join the stemmed words back into a text

# # Apply stemming to the 'safe_text' column
# df['review'] = df['review'].apply(stem_text)

In [19]:
# Display a few examples for each sentiment class

# List of unique sentiment labels (positive, negative)
sentiment_labels = df['sentiment'].unique()

# Define the number of examples to display for each class
num_examples_per_class = 3

# Loop through each sentiment class
for label in sentiment_labels:
    # Filter the DataFrame for samples with the current sentiment label
    samples = df[df['sentiment'] == label]['review'].sample(num_examples_per_class)

    # Display the sentiment label and corresponding text samples
    print(f"Sentiment Label: {label}")
    for i, text in enumerate(samples):
        print(f"Example {i + 1}: {text}\n")

Sentiment Label: positive
Example 1: me and my sister rented this movie because we were in the mood for something trashy and not so demanding to watch however the movie greatly exceeded my very low expectationsbr br it is so much more than just a representation of a century it ha very real portrayal of the character within it and most of the actor do an amazing job the different story are baked together with actual footage from the time that give it a very unique touch while watching it i really felt that i cared about what happened to the charactersbr br i would also like to give endless amount of praise to julia stile in her portrayal of katie she wa great in a way that stood outbr br i would recommend this movie to anyone

Example 2: this is the definite lars von trier movie my favorite i rank it higher than breaking the wave or the latest dancer in the dark i simply love the beauty of the picturethe framing is so original acting is wonderful a must see

Example 3: first time of see

In [20]:
# Split the train data => {train, eval}
train, eval = train_test_split(df, test_size=0.2, random_state=42, stratify=df['sentiment'])

In [21]:
train.head()

,review,sentiment
15006,i cant remember many detail about the show but...,positive
10110,of course flatley is already not exactly the i...,negative
24523,when it dvd wa released i came to market and b...,positive
2452,cashing in on the demonsmeetsclergy trend of t...,positive
15936,for those of you unfamiliar with jimmy stewart...,positive


In [22]:
eval.head()

,review,sentiment
6106,the matador is hitman movie liteif you can say...,positive
12473,br br burning paradise is a combination of neo...,positive
17907,after a long period in the space looking for t...,negative
8153,i bought this at tower record after seeing the...,negative
5431,i dont hand out ten star rating easily a movie...,positive


In [23]:
# Save splitted subsets

# Define the directory path relative to your script
output_directory = "Assets"

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Save splitted subsets
train.to_csv(os.path.join(output_directory, "train_subset.csv"), index=False)
eval.to_csv(os.path.join(output_directory, "eval_subset.csv"), index=False)


In [24]:
notebook_login()

In [25]:
dataset = load_dataset('csv',
    data_files={'train': 'Assets/train_subset.csv',
                'eval': 'Assets/eval_subset.csv'}, encoding="ISO-8859-1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [26]:
# Load the RoBERTa tokenizer
# tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [27]:
# Define a function to transform labels for the new dataset
def transform_labels(example):
    sentiment = example['sentiment']
    if sentiment == 'negative':
        num = 0
    elif sentiment == 'positive':
        num = 1
    else:
        num = -1  # Handle other values as needed
    example['sentiment'] = num
    example['labels'] = num  # Add this line to set the 'labels' field
    return example


# # Define a function to tokenize the text using the "review" column
# def tokenize_data(example):
#     return tokenizer(example['review'], padding='max_length')

def tokenize_data(example):
    # Tokenize the text and truncate to the maximum length if it's longer
    return tokenizer(example['review'], truncation=True, max_length=512, padding='max_length')


# Transform labels for the new dataset
dataset = dataset.map(transform_labels)

# Tokenize the text using the "review" column and remove other columns
remove_columns = ['review']
dataset = dataset.map(tokenize_data, batched=True)
dataset = dataset.remove_columns(remove_columns)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20000
    })
    eval: Dataset({
        features: ['sentiment', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [29]:
# training_args = TrainingArguments(
#     "movie_review",
#     num_train_epochs=5,
#     load_best_model_at_end=True,
#     evaluation_strategy="steps",
#     save_strategy="steps",
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     learning_rate=2e-5,
#     weight_decay=0.01,
#     warmup_steps=500,
#     save_total_limit=5,
#     eval_steps=100,
#     save_steps=1000,
#     push_to_hub=True,
#      metric_for_best_model="accuracy",
# )

# Define training arguments
training_args = TrainingArguments(
    output_dir="./movie_sentiment",
    num_train_epochs=5,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    load_best_model_at_end=True,
    # metric_for_best_model="eval_accuracy",
    push_to_hub=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [30]:
# Load the pre-trained RoBERTa model
# model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

# Load the pre-trained BERT model
config = BertConfig.from_pretrained("bert-base-uncased", num_labels=2)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
train_dataset = dataset['train'].shuffle(seed=10)
eval_dataset = dataset['eval'].shuffle(seed=10)

In [32]:
# Insert the code to print dataset lengths
print(f"Train dataset length: {len(train_dataset)}")
print(f"Evaluation dataset length: {len(eval_dataset)}")

Train dataset length: 20000
Evaluation dataset length: 5000


In [33]:

# Define a function to compute accuracy for your new dataset
metric = load_metric("accuracy")

def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Adjust for your specific labels: "positive" and "negative"
    references = [1 if label == "positive" else 0 for label in labels]

    return metric.compute(predictions=predictions, references=references)

In [34]:
# Define a function to compute F1 score for the new dataset
def compute_f1_score(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    f1 = f1_score(labels, preds, average="weighted")
    return {"f1_score": f1}

In [35]:
# Create a Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_accuracy,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

In [ ]:
# Launch the learning process: training
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.462600,0.458719,0.622200
1000,0.379200,0.259830,0.491800
1500,0.328200,0.345202,0.536000
2000,0.340600,0.276361,0.469400
2500,0.310700,0.341493,0.412600
3000,0.217600,0.358440,0.509600
3500,0.234700,0.339851,0.540200
4000,0.240200,0.359260,0.469400
4500,0.233900,0.331170,0.530400
5000,0.224100,0.378621,0.469400


In [ ]:
# Launch the final evaluation
eval_metrics = trainer.evaluate()

print("***** Evaluation Metrics *****")
for key, value in eval_metrics.items():
    print(f"{key}: {value}")

In [ ]:
trainer.push_to_hub()

In [ ]:
model.push_to_hub("movie_sentiment")

In [ ]:
tokenizer.push_to_hub("movie_sentiment")